XÂY DỰNG MÔ HÌNH

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Đọc dữ liệu gốc (đổi path nếu cần)
df = pd.read_csv("jena_climate_2009_2016.csv")
df = df.iloc[:50000].copy()

# Chọn các cột đặc trưng (features)
features = df[['T (degC)', 'p (mbar)', 'rho (g/m**3)']]

# Chuẩn hóa dữ liệu về khoảng [0,1]
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

print("Kích thước dữ liệu sau khi scale:", scaled_features.shape)


Kích thước dữ liệu sau khi scale: (50000, 3)


In [3]:
# Hàm tạo đầu vào X
def create_sequences(data, window_size, future):
    X = []
    for i in range(len(data) - window_size - future + 1):
        X.append(data[i : i + window_size])
    return np.array(X)

# Hàm tạo đầu ra y (dự đoán nhiệt độ)
def create_sequences_target(data, window_size, future):
    y = []
    for i in range(len(data) - window_size - future + 1):
        y.append(data[i + window_size : i + window_size + future, 0])  # cột 0 là 'T (degC)'
    return np.array(y)

# Đặt tham số nhỏ hơn để giảm thời gian
window_size = 72   
future = 24        

# Tạo dữ liệu đầu vào/đầu ra
X_seq = create_sequences(scaled_features, window_size, future)
y_seq = create_sequences_target(features.to_numpy(), window_size, future)

print("X shape:", X_seq.shape)
print("y shape:", y_seq.shape)


X shape: (49905, 72, 3)
y shape: (49905, 24)


In [4]:
# Chia dữ liệu train/test
split = int(len(X_seq) * 0.8)
X_train_ts, X_test_ts = X_seq[:split], X_seq[split:]
y_train_ts, y_test_ts = y_seq[:split], y_seq[split:]

print(f"Train shape: {X_train_ts.shape}, Test shape: {X_test_ts.shape}")
print(f"Train target: {y_train_ts.shape}, Test target: {y_test_ts.shape}")


Train shape: (39924, 72, 3), Test shape: (9981, 72, 3)
Train target: (39924, 24), Test target: (9981, 24)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Lấy thông tin đầu vào
window_size = X_train_ts.shape[1]
num_features = X_train_ts.shape[2]
future = y_train_ts.shape[1]

# Xây dựng mô hình
model = Sequential([
    LSTM(64, input_shape=(window_size, num_features), return_sequences=True),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dense(16, activation='relu'),
    Dense(future) 
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Callback dừng sớm
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('lstm_jena_model_small.keras', save_best_only=True, monitor='val_loss')
]

# Huấn luyện
history = model.fit(
    X_train_ts, y_train_ts,
    validation_split=0.2,
    epochs=30,
    batch_size=128,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/30


d:\Máy học sâu\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 17s 63ms/step - loss: 102.9164 - mae: 8.3662 - val_loss: 74.8426 - val_mae: 7.1004
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 59ms/step - loss: 77.9411 - mae: 7.3563 - val_loss: 70.0152 - val_mae: 6.8015
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 60ms/step - loss: 77.6151 - mae: 7.3426 - val_loss: 73.0268 - val_mae: 6.9935
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 15s 61ms/step - loss: 45.7876 - mae: 5.3038 - val_loss: 29.5138 - val_mae: 4.2115
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 63ms/step - loss: 26.4072 - mae: 4.0362 - val_loss: 26.2468 - val_mae: 4.0251
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 64ms/step - loss: 17.1624 - mae: 3.2568 - val_loss: 16.5713 - val_mae: 2.9847
Epoch 7/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 63ms/step - loss: 16.8715 - mae: 3.0012 - val_loss: 12.6130 - val_mae: 2.4901
Epoch 8/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 63ms/step - loss: 9.1218 - mae: 2.2879 - val_loss: 9.0183 - val_mae: 2.1317
Epoch 9/30
250/250 ━━━━━━━━━━━━━━━━━

In [6]:
from tensorflow import keras

model.save("lstm_jena_model_small.keras")
lstm_jena_model = keras.models.load_model("lstm_jena_model_small.keras")

print(" Mô hình đã được huấn luyện và lưu thành công!")


 Mô hình đã được huấn luyện và lưu thành công!
